In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

In [5]:
os.environ["SPARK_LOCAL_IP"] = "127.0.0.1" # Выпадало предупреждение при создании Спарк Сессии

In [6]:
spark = SparkSession.builder \
    .appName("ClickHouseToPySpark") \
    .config("spark.jars.packages", "com.clickhouse:clickhouse-jdbc:0.9.6") \
    .config("spark.driver.extraJavaOptions", "--add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED") \
    .config("spark.executor.extraJavaOptions", "--add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED") \
    .getOrCreate()

In [9]:
ch_options_purcases = {
    "url": "jdbc:clickhouse://localhost:9123/silver",
    "user": "user",            # логин из docker-compose
    "password": "strongpassword", # пароль
    "dbtable": "purchases",     # таблица, которую хочешь забрать
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
}

# Создаем ДФ с покупками
df_purchases = spark.read.format("jdbc").options(**ch_options_purcases).load()


In [ ]:
# Оказывается Спарк не хочет читать Array(категории) и Tuple(координаты), поэтому на этапе выгрузки подменяем саму таблицу stores
# на select запрос, в котором склеиваем (или дропаем) значения, в качестве обучения я склеил категории и дропнул координаты

In [21]:
request = "(SELECT store_id, store_name, store_network, store_description, type, arrayStringConcat(categories, ', ') as categories_str, manager_name, manager_phone, manager_email, country, city, street, house, postal_code, accept_online_orders, delivery_available, warehouse_connected, last_inventory_date, load_datetime FROM silver.stores) AS t"

In [22]:
ch_options_stores = {
    "url": "jdbc:clickhouse://localhost:9123/silver",
    "user": "user",            # логин из docker-compose
    "password": "strongpassword", # пароль
    "dbtable": request,     # таблица, которую хочешь забрать
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
    #"customSchema": f"{purchase_schema}"
}

# Создаем ДФ с магазинами
df_stores = spark.read.format("jdbc").options(**ch_options_stores).load()

In [24]:
ch_options_customers = {
    "url": "jdbc:clickhouse://localhost:9123/silver",
    "user": "user",            # логин из docker-compose
    "password": "strongpassword", # пароль
    "dbtable": "customers",     # таблица, которую хочешь забрать
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
    #"customSchema": f"{purchase_schema}"
}

# Создаем ДФ с покупателями
df_customers = spark.read.format("jdbc").options(**ch_options_stores).load()

In [26]:
ch_options_products = {
    "url": "jdbc:clickhouse://localhost:9123/silver",
    "user": "user",            # логин из docker-compose
    "password": "strongpassword", # пароль
    "dbtable": "products",     # таблица, которую хочешь забрать
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
    #"customSchema": f"{purchase_schema}"
}

# Создаем ДФ с продуктами
df_products = spark.read.format("jdbc").options(**ch_options_stores).load()